# Big Data - Aplicación de Codificación no Lineal  🧠

> **Descripción:** Realizar un código de programación para un conjunto de datos de Big Data real aplicando Codificación no Lineal. <br>
> **Autora:** Semiramís G. de la Cruz <br>

## Datos:
El dataset ha sido una adaptación de datos encontrados en [Kaggle](https://www.kaggle.com/datasets/theblackmamba31/apple-orange). Dicho dataset está compuesto por conjuntos de imágenes de naranjas y manzanas que serán un utilizados para entrenar una neurona artificial.

Se realizó un preprosesamiento previo sobre este dataset que se puede encontrar en el [repositorio en GitHub – `Practica_Codificacion_no_Lineal`](https://github.com/semilun4/Practica_Codificacion_no_Lineal/tree/main/data).

El código para la preparación se encuentra en el siguiente [script.](https://github.com/semilun4/Practica_Codificacion_no_Lineal/blob/main/syntax/script.pyy)


### Importamos los datos del repositorio

In [5]:
!wget https://raw.githubusercontent.com/semilun4/Practica_Codificacion_no_Lineal/refs/heads/main/data/testing_data.csv?token=GHSAT0AAAAAACZ3Q6KVCOOKSOURYSBMFGCOZZGM4NA
!wget https://raw.githubusercontent.com/semilun4/Practica_Codificacion_no_Lineal/refs/heads/main/data/training_data.csv?token=GHSAT0AAAAAACZ3Q6KVSZBL3WKJO5PMLONSZZGM57A

--2024-11-02 17:55:21--  https://raw.githubusercontent.com/semilun4/Practica_Codificacion_no_Lineal/refs/heads/main/data/testing_data.csv?token=GHSAT0AAAAAACZ3Q6KVCOOKSOURYSBMFGCOZZGM4NA
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11885 (12K) [text/plain]
Saving to: ‘testing_data.csv?token=GHSAT0AAAAAACZ3Q6KVCOOKSOURYSBMFGCOZZGM4NA’

testing_data.csv?to 100%[===================>]  11.61K  --.-KB/s    in 0s      

2024-11-02 17:55:21 (102 MB/s) - ‘testing_data.csv?token=GHSAT0AAAAAACZ3Q6KVCOOKSOURYSBMFGCOZZGM4NA’ saved [11885/11885]

--2024-11-02 17:55:21--  https://raw.githubusercontent.com/semilun4/Practica_Codificacion_no_Lineal/refs/heads/main/data/training_data.csv?token=GHSAT0AAAAAACZ3Q6KVSZBL3WKJO5PMLONSZZGM57A
Resolving raw.githubusercontent.com (

### Preparamos los datos

In [7]:
import pandas as pd


training_df = pd.read_csv('training_data.csv')
testing_df = pd.read_csv('testing_data.csv')

training_df

,filename,r,g,b,class
0,train/apples/apple_0.jpg,125,27,18,0
1,train/apples/apple_10.jpg,187,49,39,0
2,train/apples/apple_100.jpg,255,255,255,0
3,train/apples/apple_1000.jpg,255,255,255,0
4,train/apples/apple_1001.jpg,180,64,77,0
...,...,...,...,...,...
3233,train/oranges/orange_995.jpg,239,129,50,1
3234,train/oranges/orange_996.jpg,229,120,51,1
3235,train/oranges/orange_997.jpg,249,249,249,1
3236,train/oranges/orange_998.jpg,239,130,63,1


In [8]:
training_df['class_str'] = training_df['class'].astype('str')
training_df['hover'] = [text.split('/')[-1] for text in training_df['filename']]

testing_df['class_str'] = testing_df['class'].astype('str')
testing_df['hover'] = [text.split('/')[-1] for text in testing_df['filename']]

training_df

,filename,r,g,b,class,class_str,hover
0,train/apples/apple_0.jpg,125,27,18,0,0,apple_0.jpg
1,train/apples/apple_10.jpg,187,49,39,0,0,apple_10.jpg
2,train/apples/apple_100.jpg,255,255,255,0,0,apple_100.jpg
3,train/apples/apple_1000.jpg,255,255,255,0,0,apple_1000.jpg
4,train/apples/apple_1001.jpg,180,64,77,0,0,apple_1001.jpg
...,...,...,...,...,...,...,...
3233,train/oranges/orange_995.jpg,239,129,50,1,1,orange_995.jpg
3234,train/oranges/orange_996.jpg,229,120,51,1,1,orange_996.jpg
3235,train/oranges/orange_997.jpg,249,249,249,1,1,orange_997.jpg
3236,train/oranges/orange_998.jpg,239,130,63,1,1,orange_998.jpg


### Exploración de los datos

Verifiquemos si están balanceados

In [9]:
training_df.groupby('class').count()

,filename,r,g,b,class_str,hover
class,,,,,,
0,1566,1566,1566,1566,1566,1566
1,1672,1672,1672,1672,1672,1672


In [10]:
import plotly.express as px


fig = px.scatter_3d(
    training_df,
    x='r', y='g', z='b',
    color='class_str',
    symbol='class_str',
    color_discrete_sequence=['#be0900', '#ffb447'],
    opacity=0.5,
    hover_data=['hover']
)
fig.show()

### Creación de una neurona artificial

In [20]:
# Importamos las librerías necesarias de PyTorch
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [21]:
# Definimos la clase NeuralNet, que es nuestra red neuronal
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Creamos una secuencia de capas (red neuronal simple)
        self.model = nn.Sequential(
            nn.Linear(3, 5),   # Capa lineal que mapea de 3 (input) a 5 neuronas
            nn.ReLU(),         # Función de activación ReLU
            nn.Linear(5, 2),   # Capa lineal que mapea de 5 a 2 neuronas
            nn.ReLU(),         # Segunda función de activación ReLU
            nn.Linear(2, 1),   # Capa lineal que mapea de 2 a 1 (output)
            nn.Sigmoid()       # Función de activación Sigmoid para salida entre 0 y 1
        )

    # Definimos el paso hacia adelante
    def forward(self, x):
        out = self.model(x.view(x.size(0), 3))  # Ajustamos la entrada a tamaño (batch, 3)
        out = out.view(out.size(0), -1)  # Aplanamos el resultado de salida
        return out

Antes de entrenar los datos, procedemos a escalarlos a valores en [0, 1].

In [22]:
# Configura el dispositivo para usar GPU si está disponible, o CPU en caso contrario
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
# Normalizamos las características RGB dividiéndolas por 255
x = training_df[['r', 'g', 'b']].values / 255.
y = training_df['class'].values
y = y.reshape((len(y), 1))  # Redimensionamos y para que tenga forma (n_samples, 1)

# Convertimos los datos a tensores y los enviamos al dispositivo
tensor_x = torch.Tensor(x).to(device)
tensor_y = torch.Tensor(y).to(device)

In [29]:
# Definimos el DataLoader para facilitar el procesamiento en lotes
batch_size = 64
dataset = TensorDataset(tensor_x, tensor_y)
train_dataloader = DataLoader(dataset, batch_size=batch_size)

In [30]:
# Definimos la función de entrenamiento
def train_net(model, loss_fn, optimizer, dataloader):
    size = len(dataloader.dataset)  # Tamaño del conjunto de datos
    for batch, (x, y) in enumerate(dataloader):
        # Enviamos los datos al dispositivo
        x, y = x.to(device), y.to(device)

        # Realizamos la predicción
        pred = model(x)

        # Calculamos la pérdida entre la predicción y el valor real
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()  # Limpiamos los gradientes previos
        loss.backward()        # Calculamos los gradientes
        optimizer.step()       # Actualizamos los parámetros del modelo

    return loss.item()


In [31]:
# Inicializamos el modelo y lo enviamos al dispositivo
model = NeuralNet().to(device)

# Definimos los hiperparámetros de entrenamiento
learning_rate = 0.0002
loss_fn = nn.MSELoss()  # Función de pérdida de error cuadrático medio
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  # Optimizador SGD

# Ciclo de entrenamiento
epochs = 1000
losses = []
pbar = tqdm(range(epochs))  # Barra de progreso para ver el avance del entrenamiento
for t in pbar:
    loss = train_net(model, loss_fn, optimizer, train_dataloader)  # Entrenamos una época
    pbar.set_postfix({'Loss': loss})  # Mostramos la pérdida en la barra de progreso
    losses.append(loss)  # Guardamos la pérdida para análisis posterior

100%|██████████| 1000/1000 [01:28<00:00, 11.31it/s, Loss=0.256]


In [32]:
import numpy as np
import plotly.express as px

# Definimos el eje x como un rango igual al número de épocas (longitud de la lista de pérdidas)
eje_x = np.arange(len(losses))

# Creamos una gráfica de línea para visualizar la pérdida a lo largo de las épocas de entrenamiento
fig = px.line(
    x=eje_x,
    y=losses,
    title='Historia de entrenamiento',
    labels=dict(x='Épocas', y='Error')  # Etiquetas para los ejes
)
fig.show()


### Predicciones del modelo

In [33]:
# Preparamos los datos de una muestra para ver predicciones individuales
sample_index = 1  # Índice de muestra a verificar

# Extraemos y mostramos el color real de la muestra seleccionada
input_sample = testing_df[['r', 'g', 'b']].iloc[sample_index].values
print('Color real:', input_sample)

# Normalizamos el color dividiendo entre 255 y ajustamos la forma para el modelo
input_sample = input_sample / 255.  # Normalización
input_sample = input_sample.reshape((1, len(input_sample)))  # Ajuste de dimensión
print('Color transformado:', input_sample)

# Extraemos y mostramos la clase real de la muestra seleccionada
real_class = testing_df[['class']].iloc[sample_index].values
print('Clase real:', real_class)

# Convertimos la muestra a tensor y la pasamos al modelo para obtener una predicción
input_sample = torch.Tensor(input_sample).to(device)
predicted_class = model(input_sample).cpu().data.numpy()[0, 0]
print('Predicción del modelo:', predicted_class)


Color real: [126 178 166]
Color transformado: [[0.49411765 0.69803922 0.65098039]]
Clase real: [0]
Predicción del modelo: 0.49472427


In [34]:
import plotly.express as px

# Crea una gráfica en 3D de los datos RGB, coloreada según la clase de cada punto
fig = px.scatter_3d(
    testing_df,
    x='r', y='g', z='b',
    color='class_str',  # Colores según la clase en formato string
    symbol='class_str',  # Símbolo según la clase
    color_discrete_sequence=['#be0900', '#ffb447'],  # Paleta de colores para cada clase
    opacity=0.5,
    hover_data=['hover']  # Información adicional al pasar el mouse
)
fig.show()


In [35]:
from sklearn.metrics import accuracy_score

def get_predictions(testing_df, threshold=0.5):
    # Normalizamos las entradas dividiendo entre 255
    testing_inputs = testing_df[['r', 'g', 'b']].values / 255.
    testing_output = testing_df['class'].values  # Clases reales

    predictions = []
    for test_input in testing_inputs:
        # Ajuste de forma para que cada entrada sea un tensor de dimensión correcta
        test_input = test_input.reshape((1, len(test_input)))
        test_input = torch.Tensor(test_input).to(device)

        # Predicción usando el modelo con un umbral para clasificar
        model_output = model(test_input).cpu().data.numpy()[0, 0]
        prediction = 1 if model_output > threshold else 0
        predictions.append(prediction)

    predictions = np.array(predictions)
    return testing_output, predictions

# Calcula y muestra la precisión del modelo
testing_output, predictions = get_predictions(testing_df, threshold=0.8)
result = accuracy_score(testing_output, predictions)
print(f'Accuracy: {result * 100:.2f}%')


Accuracy: 72.79%
